In [48]:
import pandas as pd
import pickle
import json
import seaborn as sns
import pprint
pp = pprint.PrettyPrinter(depth=6)
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15.0, 5.0)
pd.set_option('display.max_columns', 120)

In [49]:
pkl_folder = "../../pkl"
csv_folder = "../../csv"

| Field          | Description                                                     | Type of variable |
|----------------|-----------------------------------------------------------------|------------------|
| fixVersions    | Search for issues that are assigned to a particular fix version | list of versions |
| issuetype      | Type of issue                                                   | Categorical      |
| project        | The project the issue belongs to                                | String           |
| resolution     | The resolution of the issue                                     | Categorical      |
| resolutiondate | The date the issue was resolved                                 | Date             |
| workratio      | calculated as workRatio = (timeSpent / originalEstimate) x 100  | Number           |
| lastViewed     | The date at which the issue was last viewed                     | Date             |
| watches        | Users watching the issue                                        | ?                |
| created        | The date of the creation of the issue                           | Date             |
| priority       | Priority assigned to the issue                                  | Categorical      |
| labels         | Labels assigned to the issue                                    | Strings          |
| versions       | The versions affected by the issue                              | List             |
| assignee       | The user the issue is assigned to                               | User             |
| status         | The status of the issue                                         | Categorical      |
| description    | The description of the issue                                    | String           |
| summary        | A summary of the issue reported                                 | String           |
| creator        | The user that created the issue                                 | User             |
| subtasks       | The subtasks of the issue                                       | List of issues   |
| reporter       | The user who reported the issue. May be the same as the creator | User             |
| duedate        | The date the issue is due to be resolved                        | Date             |

### Import tags

In [50]:
tags = pd.read_csv(f"{csv_folder}/tags.csv", index_col=0)
tags.Date = pd.to_datetime(tags.Date)
tags = tags[tags.Date > "2015-03-10"]
tags["month"] = tags.Date.dt.month
tags["year"] = tags.Date.dt.year
tags.head()

,Tag,Sha,Date,Tag_number,month,year
1,8.4.2.36762,2de1f343e3dbd9e88007b42e475512677c04eb8a,2020-08-27 14:11:31,8.4.2.36762,8,2020
2,8.4.1.35646,a0d372f704998d38856c26e28662fd7de062f873,2020-07-14 08:13:18,8.4.1.35646,7,2020
3,8.4.0.35506,74b4533439f380827d68e8c01667eeaecad89fc7,2020-07-03 07:09:27,8.4.0.35506,7,2020
4,8.3.1.34397,b77ce511a2039330aea8832ad12e86a8cfa9ff7d,2020-05-07 12:09:22,8.3.1.34397,5,2020
5,8.3.0.34182,9c58e36328ae79ad3c36c13aa253879351ce3613,2020-04-28 13:54:13,8.3.0.34182,4,2020


### Import issues

In [51]:
issues = pd.read_csv(f"{csv_folder}/issues.csv", index_col=0)
for date_field in ["created", "duedate", "lastViewed", "resolutiondate", "updated"]:
    issues[date_field] = pd.to_datetime(issues[date_field], errors="coerce")
issues = issues[issues.created > pd.to_datetime('2015-03-17 15:04:32+0000')]
issues.head()

/Library/Python/3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,6,9,12,13,17,19,22,41,42,43,60,61,82,84,89) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,issue_key,issue_id,issuetype,customfield_12130,customfield_11041,customfield_12132,customfield_12131,project,customfield_12134,customfield_12530,fixVersions,customfield_12133,customfield_12136,customfield_12532,resolution,customfield_10431,customfield_12135,customfield_12531,customfield_12138,customfield_10432,customfield_12534,customfield_12137,customfield_10433,customfield_12533,customfield_11832,customfield_11831,customfield_11833,resolutiondate,workratio,lastViewed,watches,created,customfield_11032,customfield_11033,priority,customfield_11630,customfield_11233,customfield_11036,customfield_11830,labels,customfield_11631,customfield_11038,customfield_10930,customfield_11347,customfield_10931,customfield_10932,versions,issuelinks,assignee,updated,status,components,customfield_12031,customfield_11140,customfield_12030,customfield_12033,customfield_11141,description,customfield_12032,customfield_12034,customfield_12430,customfield_11344,customfield_11343,customfield_10530,customfield_11345,customfield_10730,customfield_11337,customfield_11930,customfield_11139,customfield_11338,summary,creator,subtasks,customfield_11130,customfield_11131,reporter,customfield_11132,customfield_11133,customfield_10243,customfield_11335,customfield_11334,customfield_11730,customfield_10434,customfield_12536,customfield_12535,customfield_12139,environment,customfield_12538,customfield_10437,customfield_12537,customfield_10438,duedate,votes
0,SONAR-14169,99621,Sub-task,P3-Medium,0|i0duyn:,No root cause identified,NaN,SONAR,Not captured yet,NaN,[],Not assessed yet,Not captured yet,NaN,NaN,NaN,No action plan,NaN,Not captured yet,NaN,NaN,Not captured yet,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaT,<JIRA Watchers at 5036846000>,2020-11-19 15:56:32+01:00,NaN,NaN,Major,Developer,NaN,NaN,NaN,[],None,NaN,NaN,NaN,NaN,NaN,[],[],Michal Duda,2020-11-19 15:59:24+01:00,In Progress,[],NaN,NaN,NaN,NaN,NaN,"Due to backward compatiblity, we'll have to ke...",NaN,SonarQube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,Adapt WS to export XML definition,Michal Duda,[],NaN,NaN,Duarte Meneses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NaN,NaN,NaN,NaN,,NaN,[],2020-11-26,0
1,SONAR-14168,99609,Sub-task,P3-Medium,0|i0duvz:,No root cause identified,NaN,SONAR,Not captured yet,NaN,[],Not assessed yet,Not captured yet,NaN,Fixed,NaN,No action plan,NaN,Not captured yet,NaN,NaN,Not captured yet,NaN,NaN,NaN,NaN,NaN,2020-11-19 15:02:04+01:00,-1,NaT,<JIRA Watchers at 5037154944>,2020-11-19 12:10:36+01:00,NaN,NaN,Major,Community,NaN,NaN,NaN,[],None,NaN,NaN,NaN,NaN,NaN,[],[],Jeremy Davis,2020-11-19 15:02:04+01:00,Resolved,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,SonarQube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,[UI] Realign projects' API,Jeremy Davis,[],NaN,NaN,Jeremy Davis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NaN,NaN,NaN,NaN,,NaN,[],2020-11-26,0
2,SONAR-14167,99607,Documentation,P3-Medium,0|i0duvj:,No root cause identified,NaN,SONAR,Not captured yet,NaN,"[<JIRA Version: name='8.6', id='16435'>]",Not assessed yet,Not captured yet,NaN,NaN,NaN,No action plan,NaN,Not captured yet,NaN,NaN,Not captured yet,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaT,<JIRA Watchers at 5037155760>,2020-11-19 10:46:02+01:00,NaN,NaN,Major,Community,NaN,NaN,NaN,[],None,NaN,NaN,NaN,NaN,NaN,[],[<JIRA IssueLink: id='55366'>],Mike Birnstiehl,2020-11-19 10:46:47+01:00,Open,"[<JIRA Component: name='Documentation', id='11...",NaN,NaN,NaN,NaN,NaN,* Add insights on how to quickly analyze a pro...,NaN,SonarQube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,Improve the Azure DevOps integration page,Philippe Perrin,[],NaN,NaN,Philippe Perrin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NaN,NaN,NaN,NaN,,NaN,[],NaT,0
3,SONAR-14166,99584,Task,P3-Medium,0|i0duqf:,No root cause identified,NaN,SONAR,Not captured yet,NaN,"[<JIRA Version: name='8.6', id='16435'>]",Not assessed yet,Not captured yet,NaN,NaN,NaN,No action plan,NaN,Not captured yet,NaN,NaN,Not captured yet,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaT,<JIRA Watchers at 5037157008>,2020-11-18 14:24:30+01:00,NaN,NaN,Major,Community,NaN,NaN,NaN,['analyzer-upgrade']

We filter only issues in the timeframe in which the project was using TravisCI

### Import versions

In [52]:
versions = pd.read_csv(f"{csv_folder}/issues_versions.csv", index_col=0)
versions.head()

,issue_id,version_name,version_id
0,99500,8.5,16342
1,99361,8.5,16342
2,99004,8.5.1,16458
3,98951,8.4,15833
4,98652,8.1,15243


### Import fixVersions

In [53]:
fixversions = pd.read_csv(f"{csv_folder}/issues_fixversions.csv", index_col=0)
fixversions.head(5)

,issue_id,fixVersion_name,fixVersion_id
0,99607,8.6,16435
1,99584,8.6,16435
2,99583,8.6,16435
3,99582,8.6,16435
4,99581,8.6,16435


### Import labels

In [54]:
labels = pd.read_csv(f"{csv_folder}/issues_labels.csv", index_col=0)
labels.head()

,issue_id,label
0,99584,analyzer-upgrade
1,99583,analyzer-upgrade
2,99582,analyzer-upgrade
3,99581,analyzer-upgrade
4,99554,Events


### Import components

In [55]:
components = pd.read_csv(f"{csv_folder}/issues_components.csv", index_col=0)
components.head()

,issue_id,component_name,component_id
0,99607,Documentation,11067
1,99569,Authentication & Authorization,11086
2,99563,Web API,11100
3,99554,Web API,11100
4,99553,Issues,11096


### Import subtasks

In [56]:
substaks = pd.read_csv(f"{csv_folder}/issues_subtasks.csv", index_col=0)
substaks.head()

,issue_id,subtask_key,subtask_id
0,99379,SONAR-14134,99380
1,99379,SONAR-14135,99382
2,99379,SONAR-14136,99383
3,99379,SONAR-14137,99384
4,98843,SONAR-14098,99040


## Analysis

### Shape

In [57]:
issues.shape

(5962, 93)

In [58]:
fixversions.shape

(8628, 3)

In [59]:
versions.shape

(3351, 3)

In [60]:
labels.shape

(3019, 2)

In [61]:
substaks.shape

(884, 3)

### Columns

In [62]:
issues.columns

Index(['issue_key', 'issue_id', 'issuetype', 'customfield_12130',
       'customfield_11041', 'customfield_12132', 'customfield_12131',
       'project', 'customfield_12134', 'customfield_12530', 'fixVersions',
       'customfield_12133', 'customfield_12136', 'customfield_12532',
       'resolution', 'customfield_10431', 'customfield_12135',
       'customfield_12531', 'customfield_12138', 'customfield_10432',
       'customfield_12534', 'customfield_12137', 'customfield_10433',
       'customfield_12533', 'customfield_11832', 'customfield_11831',
       'customfield_11833', 'resolutiondate', 'workratio', 'lastViewed',
       'watches', 'created', 'customfield_11032', 'customfield_11033',
       'priority', 'customfield_11630', 'customfield_11233',
       'customfield_11036', 'customfield_11830', 'labels', 'customfield_11631',
       'customfield_11038', 'customfield_10930', 'customfield_11347',
       'customfield_10931', 'customfield_10932', 'versions', 'issuelinks',
       'assignee

In [63]:
versions.columns

Index(['issue_id', 'version_name', 'version_id'], dtype='object')

In [64]:
fixversions.columns

Index(['issue_id', 'fixVersion_name', 'fixVersion_id'], dtype='object')

In [65]:
labels.columns

Index(['issue_id', 'label'], dtype='object')

In [66]:
substaks.columns

Index(['issue_id', 'subtask_key', 'subtask_id'], dtype='object')

We have four datasets, related to each other via the primary key "issue_id". They represent different aspects of the Jira issues and will therefore analyzed in combination.

### Preliminary analysis of issue fields
We analyse the fields with no clear name or description in order to decide whether to keep them or exclude them from the analysis.

In [67]:
customfields = [col for col in issues.columns if col.startswith("customfield")]
len(customfields)

66

There are 65 customfields for which are not Jira default fields and have been defined by the project admins. We do not have descriptions for them. We let pandas describe them and manually select the ones which appear to possibly be useful to our analysis

In [68]:
issues[customfields].describe(include = ['O'])

,customfield_12130,customfield_11041,customfield_12132,customfield_12134,customfield_12133,customfield_12136,customfield_12135,customfield_12138,customfield_12137,customfield_11630,customfield_11631,customfield_11038,customfield_10930,customfield_10931,customfield_10932,customfield_12034,customfield_12430,customfield_11930,customfield_11730,customfield_12536,customfield_10437,customfield_10438
count,1081,5962,1081,1081,1081,1081,1081,1081,1081,2498,2187,2,0,0,0,1240,2,5962,108,280,5706,5962
unique,1,5962,1,1,1,1,1,1,1,5,35,2,0,0,0,1,2,1,3,1,3,1
top,P3-Medium,0|i0533b:,No root cause identified,Not captured yet,Not assessed yet,Not captured yet,No action plan,Not captured yet,Not captured yet,Community,None,MMF-29,NaN,NaN,NaN,SonarQube,https://sonarsource.productboard.com/feature-b...,{},Minutes,New,,[]
freq,1081,1,1081,1081,1081,1081,1081,1081,1081,2013,2153,1,NaN,NaN,NaN,1240,1,5962,57,280,5169,5962


We selected the ones which have more than only one unique value.
We selected the ones which have less unique values than all issues (otherwise it acts like a not informative id).
We further list their values here:

In [69]:
issues.customfield_10930.value_counts().head(10)

Series([], Name: customfield_10930, dtype: int64)

In [70]:
issues.customfield_10437.value_counts().head(10)

         5169
< 4.5     536
5.6         1
Name: customfield_10437, dtype: int64

In [71]:
issues.customfield_11730.value_counts().head(10)

Minutes    57
Hours      43
Days        8
Name: customfield_11730, dtype: int64

In [72]:
issues.customfield_11631.value_counts().head(5)

None                                                                                                                                                                                                                                 2153
To fix possible property values set during blue/green deployment, the WS POST api/system/migrate_vsts should be requested manually after the deployment. It requires system passcode or root credentials.                               1
A script needs to be executed to feed all organizations having a subscription plan from Muppet.\nA second migration (https://github.com/SonarSource/sonar-enterprise/pull/504) will be merged to set the column as not nullable.        1
Favicons/logos served for SonarCloud by the front web server can dropped                                                                                                                                                                1
Clean-up: with this ticket, PRs are officially supported in SQ. 

By filtering only customfields with more than 1 unique value and less unique values than the number of issues we obtained 4 to further analyze.
The three customfields which have been analysed did not clearly show their meaning and are therefore not included in the analysis.

In [73]:
columns_to_ignore = ["fixVersions", "issuelinks", "versions", "subtasks", "components", "labels", "watches"]

We also ignore the columns containing complex objects which we have mapped to a separate dataframe (except *watches*, *components* and *issuelinks*)

In [74]:
focus_columns = set(issues.columns) - set(customfields) - set(columns_to_ignore)
len(focus_columns)

20

### Describe

In [75]:
issues = issues[focus_columns]

In [76]:
issues.describe()

,votes,workratio,issue_id
count,5962.000000,5962.0,5962.000000
mean,0.173935,-1.0,66201.432405
std,0.957986,0.0,18762.386820
min,0.000000,-1.0,26987.000000
25%,0.000000,-1.0,51214.750000
50%,0.000000,-1.0,64366.500000
75%,0.000000,-1.0,82869.750000
max,47.000000,-1.0,99621.000000


In [77]:
issues.describe(include = ['O'])

,creator,updated,environment,resolutiondate,status,assignee,project,resolution,priority,created,issuetype,issue_key,reporter,summary,description
count,5962,5962,8,5333,5962,4713,5962,5333,5962,5962,5962,5962,5962,5962,5157
unique,84,4857,8,5274,7,58,1,11,5,5962,6,5962,85,5938,5117
top,Stas Vilchik,2017-09-12 10:47:11+02:00,MySQL 5.5.41-37.0,2020-08-18 11:01:11+02:00,Closed,Stas Vilchik,SONAR,Fixed,Major,2019-12-18 17:24:12+01:00,Improvement,SONAR-11969,Stas Vilchik,Upgrade code analyzers to latest released vers...,Template names are unique by organization. Con...
freq,815,22,1,12,5297,698,5962,4552,5537,1,3187,1,781,10,8


* Workratio seems to be not used much as 75% percentile is -1
* Votes has 75% percentile to 0 (most of the issues did not get votes) but has also a value of 192
* There are 6 issuetypes
* There are 7 statuses
* Some fields represent the same information:
    * issuetype_name and issuetype
    * assignee_key and assignee_name
    * creator_key and creator
    * resolution and resolution_name
    * priority_name and priority
    * reporter_key and reporter
    
We therefore safely remove one of each columns and keep the one in the more human readable form

In [78]:
issues.columns

Index(['creator', 'updated', 'environment', 'votes', 'lastViewed',
       'resolutiondate', 'status', 'assignee', 'workratio', 'project',
       'resolution', 'priority', 'created', 'duedate', 'issuetype',
       'issue_key', 'reporter', 'issue_id', 'summary', 'description'],
      dtype='object')

In [32]:
duplicated_columns = ["issuetype", "assignee_key", "creator_key", "resolution", "priority", "reporter_key", "resolution_id"]

In [33]:
issues = issues[set(issues.columns) - set(duplicated_columns)]

In [34]:
fixversions.describe()

,issue_id,fixVersion_id
count,8628.000000,8628.000000
mean,49158.576959,12922.841446
std,21911.712181,1455.370599
min,26471.000000,11524.000000
25%,29612.500000,11600.000000
50%,43501.500000,12781.000000
75%,64723.250000,14040.000000
max,99607.000000,16510.000000


In [35]:
versions.describe()

,issue_id,version_id
count,3351.000000,3351.000000
mean,40305.948672,12188.098478
std,19081.372449,1157.858738
min,26470.000000,11523.000000
25%,28849.500000,11556.000000
50%,30946.000000,11585.000000
75%,46216.000000,11647.000000
max,99500.000000,16458.000000


In [36]:
labels.describe()

,issue_id
count,3019.000000
mean,60489.158993
std,16434.976085
min,26487.000000
25%,51246.500000
50%,58829.000000
75%,66140.000000
max,99584.000000


In [37]:
substaks.describe()

,issue_id,subtask_id
count,884.000000,884.000000
mean,47172.642534,48065.222851
std,22079.903543,22105.395696
min,26567.000000,26526.000000
25%,29315.000000,29404.500000
50%,32441.000000,40056.500000
75%,61672.000000,61896.250000
max,99379.000000,99621.000000


We do not detect anomalies in the statistical summaries of the other datasetsm

### Missing values

In [38]:
issues.isna().sum().sort_values(ascending=False).head(15)

lastViewed        5962
environment       5954
duedate           1604
assignee          1249
description        805
resolutiondate     629
reporter             0
created              0
votes                0
summary              0
issue_key            0
updated              0
issue_id             0
project              0
workratio            0
dtype: int64

In [39]:
fixversions.isna().sum()

issue_id           0
fixVersion_name    0
fixVersion_id      0
dtype: int64

In [40]:
versions.isna().sum()

issue_id        0
version_name    0
version_id      0
dtype: int64

In [41]:
labels.isna().sum()

issue_id    0
label       0
dtype: int64

In [42]:
substaks.isna().sum()

issue_id       0
subtask_key    0
subtask_id     0
dtype: int64

* There are missing values only in the issues dataset
* All fields presenting missing values are fields which are optional in an issue. We therefore not worry about their absence even though that has to be taken into account when working with those fields.

## In depth analysis

### Workratio

In [43]:
issues.workratio.value_counts()

-1    5962
Name: workratio, dtype: int64

As seen in the statistical summary, this field seems to not be used often. It should represent a percentage of the expected work, but most of the entries have the negative value -1. We therefore decide to ignore this field.

In [44]:
issues = issues[set(issues.columns) - set(["workratio"])]

In [45]:
len(issues.columns)

16

### IssueType

In [47]:
issues.issuetype_name.unique()

,duedate,creator,updated,issue_key,environment,votes,lastViewed,reporter,resolutiondate,issue_id,status,assignee,project,created,summary,description
0,2020-11-26,Michal Duda,2020-11-19 15:59:24+01:00,SONAR-14169,NaN,0,NaT,Duarte Meneses,NaN,99621,In Progress,Michal Duda,SONAR,2020-11-19 15:56:32+01:00,Adapt WS to export XML definition,"Due to backward compatiblity, we'll have to ke..."
1,2020-11-26,Jeremy Davis,2020-11-19 15:02:04+01:00,SONAR-14168,NaN,0,NaT,Jeremy Davis,2020-11-19 15:02:04+01:00,99609,Resolved,Jeremy Davis,SONAR,2020-11-19 12:10:36+01:00,[UI] Realign projects' API,NaN
2,NaT,Philippe Perrin,2020-11-19 10:46:47+01:00,SONAR-14167,NaN,0,NaT,Philippe Perrin,NaN,99607,Open,Mike Birnstiehl,SONAR,2020-11-19 10:46:02+01:00,Improve the Azure DevOps integration page,* Add insights on how to quickly analyze a pro...
3,NaT,Malena Ebert,2020-11-18 14:25:07+01:00,SONAR-14166,NaN,0,NaT,SonarQube Technical user,NaN,99584,Open,NaN,SONAR,2020-11-18 14:24:30+01:00,Improve LITS analysis,NaN
4,NaT,Malena Ebert,2020-11-18 14:24:05+01:00,SONAR-14165,NaN,0,NaT,SonarQube Technical user,NaN,99583,Open,NaN,SONAR,2020-11-18 14:23:19+01:00,Improve Slang analysis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5965,2015-04-09,Julien Lancelot,2018-11-19 21:05:50+01:00,SONAR-6320,NaN,0,NaT,Julien Lancelot,2015-03-25 03:35:05+01:00,29597,Closed,Julien Lancelot,SONAR,2015-03-18 04:35:05+01:00,Symbol in compute,NaN
5966,2015-04-09,Julien Lancelot,2015-06-15 08:18:13+02:00,SONAR-6319,NaN,0,NaT,Julien Lancelot,2015-03-31 14:43:41+02:00,29605,Closed,Julien Lancelot,SONAR,2015-03-18 04:34:46+01:00,Feed highlighting in compute report,NaN
5967,2015-04-15,Julien Lancelot,2015-10-28 10:15:38+01:00,SONAR-6318,NaN,0,NaT,Julien Lancelot,2015-10-21 16:53:31+02:00,28763,Closed,Julien Lancelot,SONAR,2015-03-18 04:34:24+01:00,Highlighting in compute,NaN
5968,2015-04-02,Julien Lancelot,2015-06-15 08:18:13+02:00,SONAR-6317,NaN,0,NaT,Julien Lancelot,2015-03-26 10:40:48+01:00,29607,Closed,OLD - Teryk Bellahsene,SONAR,2015-03-18 04:31:30+01:00,Feed SCM in compute report,NaN


In [ ]:
issues.issuetype_name.value_counts()

In [ ]:
sns.countplot(x="issuetype_name", data=issues[["issue_id", "issuetype_name"]])

In [ ]:
issues.issuetype_name.value_counts(normalize=True)

* We see that issues can only have one of these 6 issue types.
* 54% is Improvement, followed by 22% Bug

In [ ]:
issues["created_month"] = issues.created.dt.month
issues["created_year"] = issues.created.dt.year

In [ ]:
time_group = ["created_year", "created_month"]
f, axes = plt.subplots(len(issues.issuetype_name.unique()),1, figsize=(15,20), sharex=True)
x = issues.groupby(time_group+["issuetype_name"]).size().unstack(fill_value=0).reset_index()
for i, issuetype in enumerate(issues.issuetype_name.unique()):
    ax = sns.barplot(x=x.index, y=x[issuetype], ax=axes[i])
    ax.set_xticklabels(list(x[time_group[1]].astype(int).astype(str) + "/" + x[time_group[0]].astype(int).astype(str)),
                      rotation=90)
    ax.set_ylabel(issuetype)

* New Feature, Improvement and Sub-task have decresed in number over time
* Documentation has increased after 1/2019
* Bug has been quite consistent

To note that the y-axis is not the same for the different plots. We are interested in seeing their trend overtime and using the same axis would penalize the types with fewer observations. 

### Resolution

In [ ]:
issues.resolution_name.unique()

In [ ]:
ax = sns.countplot(x="resolution_name", data=issues[["issue_id", "resolution_name"]])
ax.tick_params(axis='x', labelrotation= 60)

In [ ]:
ax = sns.countplot(issues[~(issues.resolution_name == "Fixed") & ~(issues.resolution_name == "Won't Fix")].resolution_name)
ax.tick_params(axis='x', labelrotation= 60)

In [ ]:
issues.resolution_name.value_counts(normalize=True)

* 85% of the resolution are Fixed
* 3% are duplicates (may need to be removed)

### Priority

In [ ]:
issues.priority_name.unique()

In [ ]:
ax = sns.countplot(x="priority_name", data=issues[["issue_id", "priority_name"]])

In [ ]:
issues.priority_name.isna().sum()

In [ ]:
issues.priority_name.value_counts(normalize=True)

Every issue has a priority. We have 5 values. Issue with Major priority represents the 93%

## Labels

In [ ]:
labels_merge = labels.merge(issues, on="issue_id")

**Number of labels**

In [ ]:
labels.label = labels.label.str.lower()
len(labels.label.unique())

**Number of issues with their number of labels**

In [ ]:
sns.countplot(labels_merge.groupby("issue_id").count().label)

**Top labels**

In [ ]:
vals = labels.label.value_counts().head(20)
sns.barplot(x = vals.values, y=vals.index)

* Each issue has at least one label
* Lambda label seems to be the most popular ones by far

### Versions
We filter only the versions after the project moved to TravisCI

In [ ]:
versions = versions[versions.version_name > "4.4"]
versions_merged = versions.merge(issues, on="issue_id")

In [ ]:
versions.version_name.sort_values().unique()

In [ ]:
len(versions.version_name.unique())

**Number of versions per tags**

In [ ]:
sns.countplot(versions.groupby("issue_id").count().version_name.values)

**Distribution of missing tags on Jira**

In [ ]:
#Versions on Jira have a slightly different format at times. This functions formats the versions to match Jira's
def polish_version(version):
    splits = version.split(".")
    major = splits[0]
    release = splits[1]
    if(len(splits)>2):
        modification = splits[2]
        if(modification != "0"):
            return f"{major}.{release}.{modification}"
    """if("-" in release):
        release = release.split("-")[0]"""
    return f"{major}.{release}"
releases = set(map(lambda version: polish_version(version), tags.Tag.unique()))

In [ ]:
def type_of_version(version):
    if "RC" in version: 
        return "Release candidates"
    if "M" in version:
        return "Milestones"
    return "Normal releases"
        
    
tags_not_on_jira = releases.difference(set(versions.version_name.sort_values().unique()))
tags_not_on_jira_type = list(map(lambda v: type_of_version(v), tags_not_on_jira))
github_tags_type = list(map(lambda v: type_of_version(v), tags.Tag.unique()))
sns.countplot(github_tags_type)

**Percentage of missing tags for each type**

In [ ]:
data = []
for typ in set(github_tags_type):
    data.append((typ,tags_not_on_jira_type.count(typ) / github_tags_type.count(typ) * 100))
data = pd.DataFrame(data, columns=["Release type", "% missing on jira"])
sns.barplot(x="Release type", y="% missing on jira", data=data)

In [ ]:
data

**Tags present on Jira but not on Github**

In [ ]:
set(versions.version_name.sort_values().unique()).difference(set(releases))

In [ ]:
len(set(versions.version_name.sort_values().unique()).difference(set(releases)))

In [ ]:
len(set(set(versions.version_name.sort_values().unique())).intersection(set(releases)))

* 55 versions are both present in Jira version field and on Github as tag
* 13 versions used on Jira do not correspond to a Tag on Github
* 100% of the release candidates are not used on Jira
* 96% of the milestones are not used on Jira
* Only 1.7% of normal releases are not used on Jira

**Percentage of issues with a version**

In [ ]:
len(versions.issue_id)/(len(issues.issue_id))*100

**Top versions**

In [ ]:
top_versions = versions.groupby("version_name").count().issue_id.reset_index().sort_values(by="issue_id", ascending=False)

In [ ]:
ax = sns.barplot(x="version_name", y="issue_id", data=top_versions[["issue_id", "version_name"]].head(30))

In [ ]:
top_versions.head(10).version_name.values

**Distribution of issuetypes for each version**

In [ ]:
versions_sorted = versions.sort_values(by="version_name").version_name.unique()

In [ ]:
f, axes = plt.subplots(2,1, figsize=(15,9), sharey=True)
for i in range(2):
    data = versions_merged[versions_merged.version_name.isin(versions_sorted[32*i:32*(i+1)])]
    ax = sns.countplot(ax=axes[i], x="version_name", 
                       hue="issuetype_name", data=data.sort_values(by="version_name").head(3000),
                      palette={"Bug":"r","Improvement":"orange",
                               "New Feature": "green",
                               "Task": "yellow",
                               "Sub-task": "pink",
                              "Documentation": "grey"})
    ax.tick_params(axis='x', labelrotation= 60)
    if i > 0:
        ax.get_legend().remove()

**Percentage of issuestype with a version**

In [ ]:
data = []
for issuetype in issues.issuetype_name.unique():
    data.append((issuetype, len(set(issues[issues.issuetype_name == issuetype].issue_id).intersection(set(versions.issue_id)))\
                 /len(issues[issues.issuetype_name == issuetype].issue_id)))
sns.barplot(x="issuetype", y="percentage_of_issue_with_version", data=pd.DataFrame(data, columns=["issuetype", "percentage_of_issue_with_version"]))

In [ ]:
data

* We have 63 versions we can use for our Travis analysis
* Only 17% of issues have a version
* 52% of issues with type Bug have a version

## FixVersions

In [ ]:
fixversions = fixversions[fixversions.fixVersion_name > "4.4"]
fixversions_merged = fixversions.merge(issues, on="issue_id")

In [ ]:
fixversions.fixVersion_name.unique()

In [ ]:
len(fixversions.fixVersion_name.unique())

In [ ]:
set(fixversions.fixVersion_name.unique()) - set(versions.version_name.unique())

**How many fixversions each issue has?**

In [ ]:
fixversions.groupby("issue_id").fixVersion_id.count().value_counts()

In [ ]:
sns.countplot(fixversions.groupby("issue_id").fixVersion_id.count())

**Percentage of issues with a fixversion**

In [ ]:
len(fixversions.issue_id)/(len(issues.issue_id))*100

**Top fixversions**

In [ ]:
top_fixversions = fixversions.groupby("fixVersion_name").count().issue_id.reset_index().sort_values(by="issue_id", ascending=False)

In [ ]:
ax = sns.barplot(x="fixVersion_name", y="issue_id", data=top_fixversions[["issue_id", "fixVersion_name"]].head(30))

In [ ]:
top_fixversions.head(10).fixVersion_name.values

**Distribution of issuetypes for each version**

In [ ]:
fixversions_sorted = fixversions.sort_values(by="fixVersion_name").fixVersion_name.unique()

In [ ]:
f, axes = plt.subplots(2,1, figsize=(15,9), sharey=True)
for i in range(2):
    data = fixversions_merged[fixversions_merged.fixVersion_name.isin(fixversions_sorted[32*i:32*(i+1)])]
    ax = sns.countplot(ax=axes[i], x="fixVersion_name", 
                       hue="issuetype_name", data=data.sort_values(by="fixVersion_name").head(3000),
                      palette={"Bug":"r","Improvement":"orange",
                               "New Feature": "green",
                               "Task": "yellow",
                               "Sub-task": "pink",
                              "Documentation": "grey"})
    ax.tick_params(axis='x', labelrotation= 60)
    if i > 0:
        ax.get_legend().remove()

**Percentage of issuestype with a version**

In [ ]:
data = []
for issuetype in issues.issuetype_name.unique():
    issuetype_ids = issues[issues.issuetype_name == issuetype].issue_id
    data.append((issuetype, len(set(issuetype_ids).intersection(set(fixversions.issue_id)))\
                 /len(issuetype_ids)))
sns.barplot(x="issuetype", y="percentage_of_issue_with_fixversion", data=pd.DataFrame(data, columns=["issuetype", "percentage_of_issue_with_fixversion"]))

In [ ]:
data

* We have 72 fix versions we can use for our Travis analysis (? more than versions)
* 92% of issues have a fixversion
* The issuetypes with most issues with a fixversion are Bug and New Feature

## No version issues
Ideally all bugs would be labeled with a version. Why is it not the case?

In [ ]:
no_version_issues = issues[~issues.issue_id.isin(versions.issue_id.unique())]
bug_no_version = no_version_issues[no_version_issues.issuetype_name == "Bug"]
bug_no_version.head()

**Issues with no version and their resolution type distribution**

In [ ]:
no_version_issues.resolution_name.value_counts(normalize=True)

In [ ]:
sns.countplot(no_version_issues.resolution_name)

**Percentage of issues with no versions over the years**

In [ ]:
x = no_version_issues.created_year.value_counts(normalize=True).sort_index()
sns.barplot(x.index, x.values)

**Bugs with no version resolution type distribution**

In [ ]:
bug_no_version.resolution_name.value_counts(normalize=True)

In [ ]:
sns.countplot(bug_no_version.resolution_name)

* 84% of the issues with no version have been fixed
* 86% of the issues with no version AND of type Bug have been fixed

This means that for almost 500 bug issues a solution has been found. This can possibly be exploited for identifying in which version the bug was introduced.

### Status

In [ ]:
issues.status.unique()

In [ ]:
ax = sns.countplot(x="status", data=issues[["issue_id", "status"]])

### Edition

In [ ]:
ax = sns.countplot(x="edition", data=issues)

In [ ]:
issues.edition.describe()

In [ ]:
issues.edition.isna().sum()

In [ ]:
issues[(issues.edition.isna()) & (issues.resolution_name=="Fixed") & (issues.issuetype_name=="Bug")]

### Further analysis
* Time between issue reported and closed
* Number of subtasks per issue

### Versions on Jira and not on Github (after Travis)

In [ ]:
jira_not_github = {'6.6-M1',
 '7.1',
 '7.2',
 '7.2.1',
 '7.3',
 '7.4'}

In [ ]:
focus_versions = {'7.1','7.2','7.3','7.4'}

In [ ]:
sns.countplot(versions_merged[versions_merged.version_name.isin(jira_not_github)].version_name)

In [ ]:
versions_merged[versions_merged.version_name.isin(focus_versions)].shape

In [ ]:
versions_merged.shape

In [ ]:
102/1004